<a href="https://colab.research.google.com/github/nuot/AL-Clean/blob/main/AssistantListing_ObliClean_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant Listing Obligation data cleaning project In Progress** 
<br>

Author: Nuo Tian



**Last Updated: 03/01/2021**

In [629]:
import pandas as pd
import re
data = pd.read_csv('AssistanceListings.csv')

In [630]:
obligations = data['Obligations (122)']

In [631]:
types = []
for row in obligations:
  a = re.findall(r'\((.*?)\) FY',str(row))
  if a:
    types.append(a)
  else:
    types.append('None')


In [632]:
#types_dedup = []
#for type1 in types:
    #type1 = list(dict.fromkeys(type1))
    #types_dedup.append(type1)

In [633]:
data['Obligation Types'] = types

**Checking Point** 
<br>
Done with obligation sepration
<br>
Next working on seprating the $amount for each obligation type

In [634]:
data['Obligation1'] = data['Obligation Types'].str[0]
data['Obligation2'] = data['Obligation Types'].str[1]
data['Obligation3'] = data['Obligation Types'].str[2]


In [635]:
data

,Program Title,Program Number,Popular Name (020),Federal Agency (030),Authorization (040),Objectives (050),Types of Assistance (060),Uses and Use Restrictions (070),Applicant Eligibility (081),Beneficiary Eligibility (082),Credentials/Documentation (083),Preapplication Coordination (091),Application Procedures (092),Award Procedure (093),Deadlines (094),Range of Approval/Disapproval Time (095),Appeals (096),Renewals (097),Formula and Matching Requirements (101),Length and Time Phasing of Assistance (102),Reports (111),Audits (112),Records (113),Account Identification (121),Obligations (122),Range and Average of Financial Assistance (123),Program Accomplishments (130),"Regulations, Guidelines, and Literature (140)",Regional or Local Office (151),Headquarters Office (152),Website Address (153),Related Programs (160),Examples of Funded Projects (170),Criteria for Selecting Proposals (180),Published Date,Parent Shortname,URL,Recovery,Obligation Types,Obligation1,Obligation2,Obligation3
0,Agricultural Research Basic and Applied Research,10.001,(Extramural Research),"AGRICULTURAL RESEARCH SERVICE, AGRICULTURE, DE...","{""list"":[{""act"":{""description"":""Food Security ...","To make agricultural research discoveries, eva...",PROJECT GRANTS,Not Applicable,Usually nonprofit institutions of higher educa...,Usually nonprofit institutions of higher educa...,"{""isApplicable"":false}","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Letters should be submitted to...","A peer review panel considers each proposal, e...","{""flag"":""no"",""list"":[]}",Not Applicable,None,None,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""None"",""award...","[{""code"":""program"",""isSelected"":true,""descript...","{""isApplicable"":true,""description"":""As perform...","Financial records, supporting documents, stati...",12-1400-0-1-352;,"(Project Grants) FY 19$4,704,045.00; FY 20 est...","$5,000 to $50,000. Average $20,000","{""list"":[],""isApplicable"":false}",2 CFR 200,"{""flag"":""appendix"",""description"":""See the Agri...","Deborah Watson5601 Sunnyside Avenue, MS 5110, ...",http://www.ars.usda.gov,10.207 Animal Health and Disease Research; 10....,Not Applicable.,Peer review.,"Jan 01,1965",USDA,https://beta.sam.gov/fal/2cf5bced6e734d309ddad...,No,[Project Grants],Project Grants,NaN,NaN
1,"Plant and Animal Disease, Pest Control, and An...",10.025,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Plant Protecti...",To protect U.S. agriculture from economically ...,PROJECT GRANTS,Not Applicable,"Foreign, State, local, and U.S. Territorial go...","Farmers, ranchers, agriculture producers, Stat...","{""description"":""Curriculum vitae for principal...","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Comply with E.O. 12372, \""Inte...",Applications are approved by the Administrator...,"{""flag"":""contact"",""list"":[]}",From 60 to 120 days.,None,Based on program needs and availability of ann...,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""Up to 1 year...","[{""code"":""program"",""isSelected"":false,""descrip...","{""isApplicable"":true,""description"":""N/A""}",Instruction provided in the Notice of Award. ...,12-9971-0-7-352;12-1600-0-1-352;,"(Salaries and Expenses) FY 19$288,228,684.00; ...",NaN,"{""list"":[{""fiscalYear"":2016,""description"":""Sel...","Uniform Administrative Requirements, Cost Prin...","{""flag"":""appendix"",""description"":""Consult Appe...","Eileen M. Berke,4700 River Road, Unit 55, Rive...",http://www.aphis.usda.gov/,12.100 Aquatic Plant Control; 15.611 Wildlife ...,Not Applicable.,Not Applicable.,"Jan 01,1972",USDA,https://beta.sam.gov/fal/226bb32f9354425f85946...,No,[Salaries and Expenses],Salaries and Expenses,NaN,NaN
2,Wildlife Services,10.028,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""ac

In [636]:
obli = data['Obligations (122)'].str.split(';', expand=True)

In [637]:
obli1 = data['Obligations (122)'].str.split(')', expand=True)
obli1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,(Project Grants,"FY 19$4,704,045.00; FY 20 est $6,617,316.00; ...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,(Salaries and Expenses,"FY 19$288,228,684.00; FY 20 est $298,125,341....",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,(Salaries and Expenses,"FY 19$8,832,031.00; FY 20 est $9,008,723.00; ...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,(Direct Payments with Unrestricted Use,"FY 19$12,963,091.00; FY 20 est $21,375,161.00...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,(Direct Loans,"FY 19$3,205,858,000.00; FY 20 est $3,159,314,...","FY 19$33,451,000.00; FY 20 est $65,467,000.00...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270,(Project Grants,FY 19 FY 20 FY 21 FY 18 FY 17 FY 16 - (Projec...,"FY 19$1,000,000,000.00; FY 20 est $1,000,000,...","FY 19 FY 20 FY 21 est $1,000,000,000.00; -",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2271,(Project Grants,FY 19 FY 20 FY 21 FY 18 FY 16 FY 17 - (Projec...,"FY 19$15,000,000.00; FY 20 est $15,000,000.00...","FY 19 FY 20 FY 21 est $15,000,000.00; -",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2272,(Project Grants,FY 19 FY 20 FY 21 FY 18 FY 17 FY 16 - (Projec...,FY 19$0.00; FY 20 est $0.00; FY 21 FY 18$0.00...,FY 19 FY 20 FY 21 est $0.00; -,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2273,(Project Grants,"FY 19$17,095,000.00; FY 20 est $17,200,000.00...","FY 19 FY 20 FY 21 est $3,500,000.00; -",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [638]:
mask = obli1.applymap(lambda x: x is None)
cols = obli1.columns[(mask).any()]
for col in obli1[cols]:
    obli1.loc[mask[col], col] = ''

In [639]:
df = pd.DataFrame()

i = 0
for i in range(len(obli1.columns)):
  val = []
  for row in obli1[i]:
    a = re.findall(r'FY [0-9][0-9]\$[^\]]+[.%]?',str(row))
    b = re.findall(r'FY [0-9][0-9] est \$[^\]]+[.%]?',str(row))
    if a:
      val.append(a)
    elif b:
      val.append(b)
    else:
      val.append('')
  df.insert(i, i, val, True) 


In [640]:
df = df.applymap(str)

In [641]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace("['", "")
        text = text.replace("- ']", "")
    return text

In [643]:
i = 1
while i < len(df):
  df[i] = df[i].apply(remove_punctuations)
  i +=1

KeyError: ignored

In [644]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,,"FY 19$4,704,045.00; FY 20 est $6,617,316.00; F...",,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,"FY 19$288,228,684.00; FY 20 est $298,125,341.0...",,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,"FY 19$8,832,031.00; FY 20 est $9,008,723.00; F...",,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,"FY 19$12,963,091.00; FY 20 est $21,375,161.00;...",,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,"FY 19$3,205,858,000.00; FY 20 est $3,159,314,0...","FY 19$33,451,000.00; FY 20 est $65,467,000.00;...",,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2270,,,"FY 19$1,000,000,000.00; FY 20 est $1,000,000,0...","FY 21 est $1,000,000,000.00;",,,,,,,,,,,,,,,,,,,,,,,,,,
2271,,,"FY 19$15,000,000.00; FY 20 est $15,000,000.00;...","FY 21 est $15,000,000.00;",,,,,,,,,,,,,,,,,,,,,,,,,,
2272,,,FY 19$0.00; FY 20 est $0.00; FY 21 FY 18$0.00;...,FY 21 est $0.00;,,,,,,,,,,,,,,,,,,,,,,,,,,
2273,,"FY 19$17,095,000.00; FY 20 est $17,200,000.00;...","FY 21 est $3,500,000.00;",,,,,,,,,,,,,,,,,,,,,,,,,,,


In [646]:
col1 = df[1].str.split(';', expand=True)

In [647]:
FY19 = []
Wrong = []
for i in range(len(col1[0])):
  a = re.findall(r'FY 19\$[^\]]+[.%]?', col1[0][i])
  if a :
    FY19.append(a)
  else:
    Wrong.append(col1[0][i])

In [648]:
FY19 = col1[0]
FY20 = col1[1]
FY21 = col1[2]

In [649]:
data.insert(40, 'FY19',FY19)
data.insert(41, 'FY20_est',FY20)
data.insert(42, 'FY21_est',FY21)

In [650]:
col2 = df[2].str.split(';', expand=True)
FY19_2 = col2[0]
FY20_2 = col2[1]
FY21_2 = col2[2]

data.insert(44, 'FY19_2',FY19_2)
data.insert(45, 'FY20_est_2',FY20_2)
data.insert(46, 'FY21_est_2',FY21_2)

In [651]:
col3 = df[3].str.split(';', expand=True)
FY19_3 = col3[0]
FY20_3 = col3[1]
FY21_3 = col3[2]

data.insert(48, 'FY19_3',FY19_3)
data.insert(49, 'FY20_est_3',FY20_3)
data.insert(50, 'FY21_est_3',FY21_3)

In [652]:
data

,Program Title,Program Number,Popular Name (020),Federal Agency (030),Authorization (040),Objectives (050),Types of Assistance (060),Uses and Use Restrictions (070),Applicant Eligibility (081),Beneficiary Eligibility (082),Credentials/Documentation (083),Preapplication Coordination (091),Application Procedures (092),Award Procedure (093),Deadlines (094),Range of Approval/Disapproval Time (095),Appeals (096),Renewals (097),Formula and Matching Requirements (101),Length and Time Phasing of Assistance (102),Reports (111),Audits (112),Records (113),Account Identification (121),Obligations (122),Range and Average of Financial Assistance (123),Program Accomplishments (130),"Regulations, Guidelines, and Literature (140)",Regional or Local Office (151),Headquarters Office (152),Website Address (153),Related Programs (160),Examples of Funded Projects (170),Criteria for Selecting Proposals (180),Published Date,Parent Shortname,URL,Recovery,Obligation Types,Obligation1,FY19,FY20_est,FY21_est,Obligation2,FY19_2,FY20_est_2,FY21_est_2,Obligation3,FY19_3,FY20_est_3,FY21_est_3
0,Agricultural Research Basic and Applied Research,10.001,(Extramural Research),"AGRICULTURAL RESEARCH SERVICE, AGRICULTURE, DE...","{""list"":[{""act"":{""description"":""Food Security ...","To make agricultural research discoveries, eva...",PROJECT GRANTS,Not Applicable,Usually nonprofit institutions of higher educa...,Usually nonprofit institutions of higher educa...,"{""isApplicable"":false}","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Letters should be submitted to...","A peer review panel considers each proposal, e...","{""flag"":""no"",""list"":[]}",Not Applicable,None,None,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""None"",""award...","[{""code"":""program"",""isSelected"":true,""descript...","{""isApplicable"":true,""description"":""As perform...","Financial records, supporting documents, stati...",12-1400-0-1-352;,"(Project Grants) FY 19$4,704,045.00; FY 20 est...","$5,000 to $50,000. Average $20,000","{""list"":[],""isApplicable"":false}",2 CFR 200,"{""flag"":""appendix"",""description"":""See the Agri...","Deborah Watson5601 Sunnyside Avenue, MS 5110, ...",http://www.ars.usda.gov,10.207 Animal Health and Disease Research; 10....,Not Applicable.,Peer review.,"Jan 01,1965",USDA,https://beta.sam.gov/fal/2cf5bced6e734d309ddad...,No,[Project Grants],Project Grants,"FY 19$4,704,045.00","FY 20 est $6,617,316.00","FY 21 est $7,000,000.00",NaN,,None,None,NaN,,None,None
1,"Plant and Animal Disease, Pest Control, and An...",10.025,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Plant Protecti...",To protect U.S. agriculture from economically ...,PROJECT GRANTS,Not Applicable,"Foreign, State, local, and U.S. Territorial go...","Farmers, ranchers, agriculture producers, Stat...","{""description"":""Curriculum vitae for principal...","{""environmentalImpact"":{""reports"":[{""isSelecte...","{""description"":""Comply with E.O. 12372, \""Inte...",Applications are approved by the Administrator...,"{""flag"":""contact"",""list"":[]}",From 60 to 120 days.,None,Based on program needs and availability of ann...,"{""types"":{""moe"":false,""formula"":false,""matchin...","{""awarded"":""other"",""description"":""Up to 1 year...","[{""code"":""program"",""isSelected"":false,""descrip...","{""isApplicable"":true,""description"":""N/A""}",Instruction provided in the Notice of Award. ...,12-9971-0-7-352;12-1600-0-1-352;,"(Salaries and Expenses) FY 19$288,228,684.00; ...",NaN,"{""list"":[{""fiscalYear"":2016,""description"":""Sel...","Uniform Administrative Requirements, Cost Prin...","{""flag"":""appendix"",""description"":""Consult Appe...","Eileen M. Berke,4700 River Road, Unit 55, Rive...",http://www.aphis.usda.gov/,12.100 Aquatic Plant Control; 15.611 Wildlife ...,Not Applicable.,Not Applicable.,"Jan 01,1972",USDA,https://beta.sam.gov/fal/226bb3

In [653]:
data
data.to_csv('AssistanceListingInProgress.csv')

from google.colab import files
files.download('AssistanceListingInProgress.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [654]:
#df['combined'] = df.apply(lambda row: ''.join(row),axis = 1)